In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
sns.set_palette('Set3')
from pylab import rcParams
rcParams['figure.figsize'] = 18, 8


#### Load data...

In [ ]:
train = pd.read_csv("/kaggle/input/tabular-playground-series-feb-2021/train.csv")
test = pd.read_csv("/kaggle/input/tabular-playground-series-feb-2021/test.csv")
sample_submission = pd.read_csv('../input/tabular-playground-series-feb-2021/sample_submission.csv')

### Look at data...

In [ ]:
train.tail()

In [ ]:
train.shape

# Do your stuff...

### Deleverables
- EDA
    - What's going on?
    - Show me the data...
    - Bonus: Other / Etc...
- Model
    - [RandomForestRegressor()](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html)
    - Bonus: [KNeighborsRegressor()](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html)
    - Bonus: Other / Etc...
- Evaluation
    - mean_squared_error()
    - Bonus: [roc_auc_score()](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html)
    - Bonus: Other / Etc...
- RAPIDS Bonus
    - Apply RAPIDS ([Starter Notebook](https://www.kaggle.com/tunguz/tps-feb-2021-rapids-starter))
    - Replace pandas with cuDF & sklearn with cuDF & cuML

# **EDA**

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
train.describe()

In [ ]:
test.describe()

In [ ]:
sample_submission.sample(5)

In [ ]:
# Catgorical column describtion 
train.describe(exclude=[np.number])

In [ ]:
sns.displot(train.target).set(title='Distripution of Target');
plt.show();

In [ ]:
corr = train.corr()
corr.style.background_gradient(cmap='Blues')


In [ ]:
sns.boxplot(data = train)

# **Prepar Data to modling**

In [ ]:
train.head()

In [ ]:
train.columns

In [ ]:
%%capture
!pip install category-encoders
import category_encoders as ce

# Ordinally Encoded Train Dataframe
encoder = ce.OrdinalEncoder()
train_encode = encoder.fit_transform(train)

In [ ]:
train_encode.head()

In [ ]:
# Ordinally Encoded Test Dataframe 
encoder = ce.OrdinalEncoder()
test_encode = encoder.fit_transform(test)

In [ ]:
test_encode.head()

# **Spilt Data**

In [ ]:
from sklearn.model_selection import train_test_split
# select split parameter
x = train_encode.drop(['target'], axis=1)
y = train_encode.target

In [ ]:
# splitting into train and test sets for both parameters
x_train, x_test ,y_train , y_test = train_test_split(x,y, test_size=0.7, random_state= 42)

# **Models**

# 1.**Random Forest Regressor**

In [ ]:
from sklearn.ensemble import RandomForestRegressor


rfr = RandomForestRegressor()

In [ ]:
rfr.fit(x_train, y_train)

In [ ]:
rfr_preds = rfr.predict(x_test)

rfr_preds

In [ ]:
# Evaluate MSE
from sklearn.metrics import mean_squared_error

mean_squared_error(y_test, rfr_preds)

In [ ]:
from sklearn.metrics import mean_absolute_error

mean_absolute_error(y_test, rfr_preds)

# 2.**KNeighbors Regressor**

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor()

In [ ]:
knn.fit(x_train, y_train)

In [ ]:
knn_preds = knn.predict(x_test)

knn_preds

In [ ]:
# Evaluate MSE
mean_squared_error(y_test, knn_preds)

In [ ]:
# Evaluate MAE
mean_absolute_error(y_test, knn_preds)

# 3.**LinearRegression**

In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()

In [ ]:
lr.fit(x_train, y_train)

In [ ]:
lr_preds = lr.predict(x_test)

lr_preds

In [ ]:
# Evaluate MSE
mean_squared_error(y_test, lr_preds)

In [ ]:
# Evaluate MAE
mean_absolute_error(y_test, lr_preds)

# **Pridict on Test Data**

In [ ]:
test_preds = rfr.predict(test_encode)
test_preds

In [ ]:
sample_submission['target'] = test_preds
sample_submission.to_csv('submission.csv', index=False)